## Load Required Packages and Define Functions

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import metrics

import numpy as np
import pandas as pd
import pickle
import time

def get_points(file):
    '''load matlab style file'''
    mat = scipy.io.loadmat(file)
    return mat[list(mat.keys())[3]]

def pickle_save(filename, content):
    '''save the file into python pickle object under output folder'''
    with open('../output/%s.pkl'%filename, 'wb') as f:
        pickle.dump(content, f)
        
def pickle_open(filename):
    '''load the pickle file'''
    with open('../output/%s.pkl'%filename, 'rb') as f:
        content = pickle.load(f)
    return content

## XGBoost

### 1. Load data and train-test split

In [2]:
# load data from pickle oject
fiducial_pt_full = pickle_open('fiducial_pt_full')
label_full = pickle_open('label_full')

## Note: randomly split into training & test set
X_train, X_test, y_train, y_test = train_test_split(fiducial_pt_full, label_full, test_size=0.2, random_state=42)


### 2. Train/test feature extraction

Here, use pairwise distances as features.

In [3]:
from sklearn.metrics import pairwise_distances

# extract pairwise distance as features (78*77/2=3003 features)
# nrow=number of records of the dataset; ncol=3003
start_time = time.time()
feature_train = np.stack((pairwise_distances(X_train[i])[np.triu_indices(78, k = 1)] for i in range(X_train.shape[0])))
print('XGBoost training feature extraction takes %s seconds.'%round((time.time()-start_time),3))

start_time = time.time()
feature_test = np.stack((pairwise_distances(X_test[i])[np.triu_indices(78, k = 1)] for i in range(X_test.shape[0])))
print('XGBoost test feature extraction takes %s seconds.'%round((time.time()-start_time),3))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


XGBoost training feature extraction takes 0.633 seconds.
XGBoost test feature extraction takes 0.146 seconds.


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### A balanced test data splitted from test data

In [4]:
emotion_1 = y_test[y_test == 1]
emotion_0 = y_test[y_test== 0]
feature_1 = feature_test[y_test==1]
feature_0 = feature_test[y_test==0]
bal_feature = np.concatenate((feature_1[0:130],feature_0[0:130]),axis=0)
bal_y = np.concatenate((emotion_1[0:130],emotion_0[0:130]),axis=0)

### 3. Train XGBoost model with training features and labels

* scale_pos_weight = total_negative_examples / total_positive_examples
* negative: majority; positive: minority

In [3]:
# majority is label 0; minority is label 1

# estimate scale_pos_weight value
estimate = np.sum(y_train==0)/np.sum(y_train==1)
print('Estimate: %.3f' % estimate)

Estimate: 4.229


In [84]:
def fit_xgb(algo, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=25):
    
    if useTrainCV:
        xgb_param = algo.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb_param['n_estimators'], 
                          nfold=cv_folds,stratified=True, metrics='auc', 
                          early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        algo.set_params(n_estimators=cvresult.shape[0])
    
    # fit the algorithm with cv selected parameters on training data
    algo.fit(X_train, y_train, eval_metric='auc')
    
    # predict on training data
    pred_train = algo.predict(X_train)
    predprob_train = algo.predict_proba(X_train)[:,1]
    
    # model report
    print('\nModel Report')
    print('Accuracy (train): %.4f'%metrics.accuracy_score(y_train, pred_train))
    print('AUC Score (train): %.4f'%metrics.roc_auc_score(y_train, predprob_train))


In [85]:
# use cross validation and early_stopping_rounds to decide n_estimators
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1500,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=4,
 seed=27)


time_start_xgb1 = time.time()

fit_xgb(xgb1, feature_train, y_train)

time_end_xgb1 = time.time()
print('Training model takes {:.2f} s'.format(time_end_xgb1-time_start_xgb1))


Model Report
Accuracy (train): 1.0000
AUC Score (train): 1.0000
Training model takes 221.17 s


In [86]:
start_time = time.time()

pred_test = xgb1.predict(feature_test)
predprob_test = xgb1.predict_proba(feature_test)[:,1]

print('Predicting on test set takes %.4f seconds'%(time.time()-start_time))
print('Accuracy (test): %.4f'%metrics.accuracy_score(y_test, pred_test))
print('Precision (test): %.4f'%metrics.precision_score(y_test, pred_test))
print('Recall (test): %.4f'%metrics.recall_score(y_test, pred_test))
print('AUC Score (test): %.4f'%metrics.roc_auc_score(y_test, predprob_test))


Predicting on test set takes 0.0559 seconds
Accuracy (test): 0.8300
Precision (test): 0.7342
Recall (test): 0.4173
AUC Score (test): 0.8478


In [87]:
# check parameters
xgb1.get_xgb_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 189,
 'nthread': 4,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 4,
 'seed': 27,
 'subsample': 0.8,
 'verbosity': 1}

### 4. Parameter tuning
### 4.1 Tune max_depth and min_child_weight

In [88]:
param_test1 = {
 'max_depth':range(3,8,2),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(
    estimator = XGBClassifier(learning_rate=0.1, n_estimators=189, max_depth=5,
                              min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                              objective= 'binary:logistic', nthread=4, scale_pos_weight=4, seed=27),
    param_grid = param_test1, scoring='roc_auc',n_jobs=-1,cv=5)


time_start_gs1 = time.time()

gsearch1.fit(feature_train, y_train)

time_end_gs1 = time.time()
print('Tuning time cost {:.2f} s'.format(time_end_gs1-time_start_gs1))

Tuning time cost 1115.06 s


In [89]:
# report the best configuration
print("Best: %f using %s" % (gsearch1.best_score_, gsearch1.best_params_))

# report all configurations
means = gsearch1.cv_results_['mean_test_score']
stds = gsearch1.cv_results_['std_test_score']
params = gsearch1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.832756 using {'max_depth': 5, 'min_child_weight': 5}
0.827208 (0.007249) with: {'max_depth': 3, 'min_child_weight': 1}
0.823122 (0.007797) with: {'max_depth': 3, 'min_child_weight': 3}
0.819270 (0.005282) with: {'max_depth': 3, 'min_child_weight': 5}
0.830624 (0.005024) with: {'max_depth': 5, 'min_child_weight': 1}
0.828628 (0.003036) with: {'max_depth': 5, 'min_child_weight': 3}
0.832756 (0.007105) with: {'max_depth': 5, 'min_child_weight': 5}
0.827725 (0.005165) with: {'max_depth': 7, 'min_child_weight': 1}
0.822672 (0.004134) with: {'max_depth': 7, 'min_child_weight': 3}
0.832728 (0.006745) with: {'max_depth': 7, 'min_child_weight': 5}


### 4.2 Tune Gamma

In [90]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}


gsearch2 = GridSearchCV(
    estimator = XGBClassifier(learning_rate=0.1, n_estimators=189, max_depth=5,
                              min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                              objective= 'binary:logistic', nthread=4, scale_pos_weight=4, seed=27),
    param_grid = param_test2, scoring='roc_auc',n_jobs=-1,cv=5)


time_start_gs2 = time.time()

gsearch2.fit(feature_train, y_train)

time_end_gs2 = time.time()
print('Tuning time cost {:.2f} s'.format(time_end_gs3-time_start_gs3))

Tuning time cost 939.01 s


In [91]:
# report the best configuration
print("Best: %f using %s" % (gsearch2.best_score_, gsearch2.best_params_))

# report all configurations
means = gsearch2.cv_results_['mean_test_score']
stds = gsearch2.cv_results_['std_test_score']
params = gsearch2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.833379 using {'gamma': 0.2}
0.832756 (0.007105) with: {'gamma': 0.0}
0.827531 (0.005324) with: {'gamma': 0.1}
0.833379 (0.003730) with: {'gamma': 0.2}
0.832165 (0.003866) with: {'gamma': 0.3}
0.830586 (0.005902) with: {'gamma': 0.4}


### 4.3 Tune regularization parameters

In [92]:
param_test3 = {
 'reg_alpha':[0.005, 0.01, 0.05, 1],
 'reg_lambda':[0.05, 0.1, 1]
}

gsearch3 = GridSearchCV(
    estimator = XGBClassifier(learning_rate=0.1, n_estimators=189, max_depth=5,
                              min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8,
                              objective= 'binary:logistic', nthread=4, scale_pos_weight=4, seed=27),
    param_grid = param_test3, scoring='roc_auc',n_jobs=-1,cv=5)


time_start_gs3 = time.time()

gsearch3.fit(feature_train, y_train)

time_end_gs3 = time.time()
print('Tuning time cost {:.2f} s'.format(time_end_gs3-time_start_gs3))

Tuning time cost 1419.89 s


In [93]:
# report the best configuration
print("Best: %f using %s" % (gsearch3.best_score_, gsearch3.best_params_))

# report all configurations
means = gsearch3.cv_results_['mean_test_score']
stds = gsearch3.cv_results_['std_test_score']
params = gsearch3.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.832915 using {'reg_alpha': 1, 'reg_lambda': 1}
0.824504 (0.004240) with: {'reg_alpha': 0.005, 'reg_lambda': 0.05}
0.829220 (0.010068) with: {'reg_alpha': 0.005, 'reg_lambda': 0.1}
0.832020 (0.007155) with: {'reg_alpha': 0.005, 'reg_lambda': 1}
0.823307 (0.006179) with: {'reg_alpha': 0.01, 'reg_lambda': 0.05}
0.826827 (0.004085) with: {'reg_alpha': 0.01, 'reg_lambda': 0.1}
0.829055 (0.005793) with: {'reg_alpha': 0.01, 'reg_lambda': 1}
0.831763 (0.014205) with: {'reg_alpha': 0.05, 'reg_lambda': 0.05}
0.827051 (0.003367) with: {'reg_alpha': 0.05, 'reg_lambda': 0.1}
0.828453 (0.006281) with: {'reg_alpha': 0.05, 'reg_lambda': 1}
0.828249 (0.002748) with: {'reg_alpha': 1, 'reg_lambda': 0.05}
0.828960 (0.002081) with: {'reg_alpha': 1, 'reg_lambda': 0.1}
0.832915 (0.005444) with: {'reg_alpha': 1, 'reg_lambda': 1}


In [94]:
param_test4 = {
 'reg_alpha':[0.5, 1, 3, 5],
 'reg_lambda':[0.5, 1, 3, 5]
}

gsearch4 = GridSearchCV(
    estimator = XGBClassifier(learning_rate=0.1, n_estimators=189, max_depth=5,
                              min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8,
                              objective= 'binary:logistic', nthread=4, scale_pos_weight=4, seed=27),
    param_grid = param_test4, scoring='roc_auc',n_jobs=-1,cv=5)


time_start_gs4 = time.time()

gsearch4.fit(feature_train, y_train)

time_end_gs4 = time.time()
print('Tuning time cost {:.2f} s'.format(time_end_gs4-time_start_gs4))


/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Tuning time cost 1985.69 s


In [95]:
# report the best configuration
print("Best: %f using %s" % (gsearch4.best_score_, gsearch4.best_params_))

# report all configurations
means = gsearch4.cv_results_['mean_test_score']
stds = gsearch4.cv_results_['std_test_score']
params = gsearch4.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.833544 using {'reg_alpha': 1, 'reg_lambda': 5}
0.831550 (0.003642) with: {'reg_alpha': 0.5, 'reg_lambda': 0.5}
0.827960 (0.004573) with: {'reg_alpha': 0.5, 'reg_lambda': 1}
0.824485 (0.005804) with: {'reg_alpha': 0.5, 'reg_lambda': 3}
0.822690 (0.007957) with: {'reg_alpha': 0.5, 'reg_lambda': 5}
0.831430 (0.001636) with: {'reg_alpha': 1, 'reg_lambda': 0.5}
0.832915 (0.005444) with: {'reg_alpha': 1, 'reg_lambda': 1}
0.832098 (0.005856) with: {'reg_alpha': 1, 'reg_lambda': 3}
0.833544 (0.002235) with: {'reg_alpha': 1, 'reg_lambda': 5}
0.825784 (0.004279) with: {'reg_alpha': 3, 'reg_lambda': 0.5}
0.824243 (0.004478) with: {'reg_alpha': 3, 'reg_lambda': 1}
0.826299 (0.006927) with: {'reg_alpha': 3, 'reg_lambda': 3}
0.825894 (0.006178) with: {'reg_alpha': 3, 'reg_lambda': 5}
0.827201 (0.002708) with: {'reg_alpha': 5, 'reg_lambda': 0.5}
0.824550 (0.005536) with: {'reg_alpha': 5, 'reg_lambda': 1}
0.829269 (0.004529) with: {'reg_alpha': 5, 'reg_lambda': 3}
0.825618 (0.007652) with: {'r

In [96]:
pred_test = gsearch4.best_estimator_.predict(feature_test)
predprob_test = gsearch4.best_estimator_.predict_proba(feature_test)[:,1]
print('Accuracy (test): %.4f'%metrics.accuracy_score(y_test, pred_test))
print('AUC Score (test): %.4f'%metrics.roc_auc_score(y_test, predprob_test))


Accuracy (test): 0.8333
AUC Score (test): 0.8403


In [97]:
xgb3 = XGBClassifier(
 learning_rate =0.05,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=5,
 gamma=0.4,
 reg_alpha=1,
 reg_lambda=5,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=4,
 seed=27)

time_start_xgb3 = time.time()

fit_xgb(xgb3, feature_train, y_train)

time_end_xgb3 = time.time()
print('Training model takes {:.2f} s'.format(time_end_xgb3-time_start_xgb3))



Model Report
Accuracy (train): 0.9988
AUC Score (train): 1.0000
Training mode takes 234.83 s


In [98]:
start_time = time.time()

pred_test = xgb3.predict(feature_test)
predprob_test = xgb3.predict_proba(feature_test)[:,1]

print('Predicting on test set takes %.4f seconds'%(time.time()-start_time))
print('Accuracy (test): %.4f'%metrics.accuracy_score(y_test, pred_test))
print('Precision (test): %.4f'%metrics.precision_score(y_test, pred_test))
print('Recall (test): %.4f'%metrics.recall_score(y_test, pred_test))
print('AUC Score (test): %.4f'%metrics.roc_auc_score(y_test, predprob_test))


Predicting on test set takes 0.0555 seconds
Accuracy (test): 0.8333
Precision (test): 0.7097
Recall (test): 0.4748
AUC Score (test): 0.8342


In [104]:
xgb3

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=None, n_estimators=194, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=1, reg_lambda=5, scale_pos_weight=4, seed=27,
              silent=None, subsample=0.8, verbosity=1)

### 4.4 Tune weights

In [99]:
weights = [1, 5, 10, 20, 50, 70, 90, 100, 1000]
param_grid = {'scale_pos_weight':weights}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid = GridSearchCV(estimator=xgb3, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

time_start_gs = time.time()

grid_result = grid.fit(feature_train, y_train)

time_end_gs = time.time()
print('Tuning time cost {:.2f} s'.format(time_end_gs-time_start_gs))


Tuning time cost 1216.86 s


In [100]:
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.841085 using {'scale_pos_weight': 10}
0.840166 (0.017949) with: {'scale_pos_weight': 1}
0.837953 (0.012144) with: {'scale_pos_weight': 5}
0.841085 (0.011429) with: {'scale_pos_weight': 10}
0.833068 (0.018257) with: {'scale_pos_weight': 20}
0.819581 (0.013654) with: {'scale_pos_weight': 50}
0.810718 (0.017646) with: {'scale_pos_weight': 70}
0.811681 (0.016839) with: {'scale_pos_weight': 90}
0.809549 (0.018111) with: {'scale_pos_weight': 100}
0.771769 (0.023255) with: {'scale_pos_weight': 1000}


In [101]:
pred_train = grid_result.best_estimator_.predict(feature_train)
predprob_train = grid_result.best_estimator_.predict_proba(feature_train)[:,1]
print('Accuracy (train): %.4f'%metrics.accuracy_score(y_train, pred_train))
print('AUC Score (train): %.4f'%metrics.roc_auc_score(y_train, predprob_train))

Accuracy (train): 0.9896
AUC Score (train): 1.0000


In [105]:
pred_test = grid_result.best_estimator_.predict(feature_test)
predprob_test = grid_result.best_estimator_.predict_proba(feature_test)[:,1]
print('Accuracy (test): %.4f'%metrics.accuracy_score(y_test, pred_test))
print('Precision (test): %.4f'%metrics.precision_score(y_test, pred_test))
print('Recall (test): %.4f'%metrics.recall_score(y_test, pred_test))
print('AUC Score (test): %.4f'%metrics.roc_auc_score(y_test, predprob_test))

Accuracy (test): 0.8133
Precision (test): 0.6031
Recall (test): 0.5683
AUC Score (test): 0.8318


In [103]:
grid_result.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=None, n_estimators=194, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=1, reg_lambda=5, scale_pos_weight=10, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [106]:
metrics.confusion_matrix(y_test, pred_test)

array([[409,  52],
       [ 60,  79]])

### After tuning, learning_rate=0.05, gamma=0.4, max_step=5, min_child_weight=5, n_estimators=194, reg_alpha=1, reg_lambda=5, scale_pos_weight=10. 

In [109]:
# store the trained model
pickle_save('xgb_v1', grid_result.best_estimator_)

In [5]:
xgb_v1 = pickle_open('xgb_v1')

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
def clf_metrics(y_true, y_pred, y_score):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_score)
    
    df = pd.DataFrame({'accuracy':[accuracy],'precision': [precision], 'recall': [recall], 'auc':[auc]})
    print(df)

In [10]:
# Tuned XGBoost performance
pred_train = xgb_v1.predict(feature_train)
score_train = xgb_v1.predict_proba(feature_train)[:,1]
print('Training set:')
clf_metrics(y_train, pred_train, score_train)
print('\n')

pred_test = xgb_v1.predict(feature_test)
score_test = xgb_v1.predict_proba(feature_test)[:,1]
print('Test set:')
clf_metrics(y_test, pred_test, score_test)
print('\n')

pred_test = xgb_v1.predict(bal_feature)
score_test = xgb_v1.predict_proba(bal_feature)[:,1]
print('Balanced test set:')
clf_metrics(bal_y, pred_test, score_test)

Training set:
   accuracy  precision  recall  auc
0  0.989583   0.948347     1.0  1.0


Test set:
   accuracy  precision    recall      auc
0  0.813333   0.603053  0.568345  0.83177


Balanced test set:
   accuracy  precision    recall      auc
0  0.753846   0.902439  0.569231  0.86503
